In [1]:
import os
import warnings
import numpy
import pandas
import geopandas
import h3
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import pickle

from tqdm import tqdm_notebook
from cartoframes.viz import *

warnings.filterwarnings('ignore')
warnings.simplefilter  ('ignore')

%matplotlib inline
sns.set()

pandas.set_option('display.max_rows',     500)
pandas.set_option('display.max_columns',  500)
pandas.set_option('display.width',       1000)
pandas.set_option('display.max_colwidth', 100)

In [2]:
from gistools.utils import to_timestr, datetime2str, add_to
from gistools.gmapsro import FleetRoutingEntity, FleetRouting, read_csv
from gistools.geometry import to_geo, to_shapely, decode_polyline

In [3]:
DEPOT_ID = 'S_OLO'
DEPOT_LABEL = 'HUB Olomouc'

In [4]:
shipments = read_csv('_dataset/shipments.csv')

shipments['tags'] = shipments['tags'].fillna(value='') 
shipments['forbidden_tags'] = shipments['forbidden_tags'].fillna(value='') 

shipments = shipments.rename(columns={'name': 'shipment_name', 'visit_type': 'shipment_type'})

print('{}'.format(len(shipments)))
shipments.head()

408


,id,shipment_name,location_id,org_name,depot_id,depot_name,shipment_type,period,date_of_delivery,time_window_start,time_window_end,duration,floor_pallets,tags,forbidden_tags,allowed_vehicles,input_address,input_postal_code,input_city,input_country_code,formatted_address,street_number,street,address,city,postal_code,admin_area_level_2,admin_area_level_1,country,country_code,place_id,place_name,place_type,place_main_type,place_URL,website,phone,email,monday,tuesday,wednesday,thursday,friday,saturday,sunday,location_type,confidence,maps_URL,longitude,latitude,geometry,gmapsro_id,start_datetime,end_datetime,start_timestamp,end_timestamp
0,14387481_D,vysočanská_19000,vysočanská_19000,FM Logistic,S_TUC,FM Česká s.r.o.,delivery,2023-11-13,2023-11-13,08:00:00,14:00:00,15,50,S_TUC,NAVES,5SA 7826|7AP 5246|7AP 5247|1SF 6400|2AL 1348|3SL 5180|4AA 5279|4SR 7390|5C7 0653|6B8 8048|3SP 84...,Vysočanská,190 00,Praha,CZ,"Vysočanská 20/382, 190 00 Praha 9-Prosek, Czechia",20/382,Vysočanská,20/382 Vysočanská,NaN,190 00,Hlavní město Praha,Hlavní město Praha,Czechia,cz,ChIJQ48GmL7sC0cRy_0AacxWA3I,Billa,"['supermarket', 'grocery_or_supermarket', 'store', 'point_of_interest', 'food', 'establishment']",supermarket,https://maps.google.com/?cid=8215505581188906443,http://www.billa.cz/,+420 800 153 976,NaN,07:00-23:00,07:00-23:00,07:00-23:00,07:00-23:00,07:00-23:00,07:00-23:00,08:00-23:00,ROOFTOP,0.66,https://www.google.com/maps/search/?api=1&query=50.119825%2C14.499439&query_place_id=ChIJQ48GmL7...,14.499439,50.119825,POINT (14.49944 50.11982),0,2023-11-13 08:00:00,2023-11-13 14:00:00,1699858800,1699880400
1,14390258_D,severni253_25219,severni253_25219,FM Logistic,S_TUC,FM Česká s.r.o.,delivery,2023-11-13,2023-11-13,06:00:00,18:00:00,15,100,S_TUC,,5SA 7826|2TI 7458|3SP 2412|7AP 5246|7AP 5247|1SF 6400|2AL 1348|3SL 5180|4AA 5279|4SR 7390|5C7 06...,SEVERNI 253,252 19,CHRASTANY,CZ,"Severní 253, 252 19 Chrášťany-Rudná u Prahy, Czechia",253,Severní,253 Severní,Chrášťany,252 19,Praha-západ,Středočeský kraj,Czechia,cz,ChIJycmKlj-_C0cRWQ12-AbCQLI,Rohlík.cz - warehouse Chrášťany,"['storage', 'grocery_or_supermarket', 'food', 'point_of_interest', 'store', 'establishment']",storage,https://maps.google.com/?cid=12844479472454733145,http://rohlik.cz/,+420 800 730 740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROOFTOP,0.68,https://www.google.com/maps/search/?api=1&query=50.051750%2C14.256753&query_place_id=ChIJycmKlj-...,14.256753,50.051750,POINT (14.25675 50.05175),1,2023-11-13 06:00:00,2023-11-13 18:00:00,1699851600,1699894800
2,14390261_D,17.listopadu_70800,17.listopadu_70800,FM Logistic,S_OLO,HUB Olomouc,delivery,2023-11-13,2023-11-13,06:00:00,18:00:00,15,100,S_OLO,,4S5 6332|7M7 3641|8AL 2282|7AF 6001|7Z2 3406|1BS 2104|4M5 6808|6S1 0664|2M7 1261|6M8 2032|6M9 91...,17. listopadu,708 00,Ostrava,CZ,"R574+93, 708 00 Ostrava-Poruba, Czechia",NaN,R574+93 708 00 Ostrava-Poruba Czechia,R574+93 708 00 Ostrava-Poruba Czechia,Ostrava 8,708 00,Ostrava-City District,Moravian-Silesian Region,Czechia,cz,ChIJrbeS837nE0cRZxaUbRdvOC4,Rohlik.cz Sklad Ostrava,"['storage', 'point_of_interest', 'establishment']",storage,https://maps.google.com/?cid=3330534070853834343,http://www.rohlik.cz/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GEOMETRIC_CENTER,0.68,https://www.google.com/maps/search/?api=1&query=49.813385%2C18.155238&query_place_id=ChIJrbeS837...,18.155238,49.813385,POINT (18.15524 49.81339),2,2023-11-13 06:00:00,2023-11-13 18:00:00,1699851600,1699894800
3,14390592_D,masarykovonam.2799_53002,masarykovonam.2799_53002,FM Logistic,S_TUC,FM Česká s.r.o.,delivery,2023-11-13,2023-11-13,08:00:00,08:30:00,15,100,S_TUC,,5SA 7826|2TI 7458|3SP 2412|7AP 5246|7AP 5247|1SF 6400|2AL 1348|3SL 5180|4AA 5279|4SR 7390|5C7 06...,Masarykovo nam. 2799,530 02,Pardubice,CZ,"Masarykovo nám. 2799/2799, 530 02 Pardubice I, Czechia",2799,Masarykovo náměstí,2799 Masarykovo náměstí,NaN,530 02,Pardubice,Pardubický kraj,Czechia,cz,ChIJz9Qx5b_MDUcRzFlxHnDKnms,Yves Rocher,"['point_of_interest', 'store', 'establishment']",point_of_i

In [5]:
shipments.loc[shipments[shipments['shipment_type'] == 'pickup'  ].index, 'shipment_type'] = 'pickup_and_depot_drop_off'
shipments.loc[shipments[shipments['shipment_type'] == 'delivery'].index, 'shipment_type'] = 'depot_pickup_and_delivery'

In [6]:
shipments.value_counts('shipment_type')

shipment_type
depot_pickup_and_delivery    403
pickup_and_depot_drop_off      5
Name: count, dtype: int64

In [7]:
depots = read_csv('_dataset/depots.csv')

print('{}'.format(len(depots)))
depots.head()

2


,depot_id,depot_name,org_name,formatted_address,maps_url,longitude,latitude,geometry,gmapsro_id
0,S_TUC,FM Česká s.r.o.,FM Logistic,"Ke Spejcharu 351, 252 67 Tuchoměřice","https://maps.google.com/maps?ll=50.136635,14.276553&z=17&t=m&hl=en&gl=US&mapclient=apiv3&cid=183...",14.273747,50.137567,POINT (14.27375 50.13757),0
1,S_OLO,HUB Olomouc,FM Logistic,"Na Statkách 6-81, 783 01 Olomouc","https://maps.google.com/maps?ll=49.567893,17.205674&z=17&t=m&hl=en&gl=US&mapclient=apiv3&cid=118...",17.206987,49.568437,POINT (17.20699 49.56844),1


In [8]:
vehicles = read_csv('_dataset/vehicles.csv')

print('{}'.format(len(vehicles)))
vehicles.head()

173


,vehicle_id,vehicle_name,depot_id,depot_name,org_name,cost_per_kilometer,fixed_cost,route_duration_limit,route_duration_limit_soft_max,route_duration_limit_cost_per_hour_after_soft_max,start_datetime,end_datetime,start_timestamp,end_timestamp,depot_service_time,allowed_overtime,floor_pallets,S_TUC,S_OLO,FRIGO,NAVES,TESCO,CELO,SOLO,BOHEMIA_ONLY,DODAVKA,KINDL,longitude,latitude,geometry,gmapsro_id
0,5SJ 2795,5SJ 2795-CERMAK,S_TUC,FM Česká s.r.o.,FM Logistic,30.0,7500,540,0,NaN,2023-11-12 22:00:00,2023-11-13 23:59:00,1699822800,1699916340,45,0,3300,S_TUC,NaN,FRIGO,NAVES,NaN,NaN,NaN,NaN,NaN,NaN,14.273747,50.137567,POINT (14.27375 50.13757),0
1,3SF 8386,3SF 8386-CAJA,S_TUC,FM Česká s.r.o.,FM Logistic,30.0,7500,540,0,NaN,2023-11-12 22:00:00,2023-11-13 23:59:00,1699822800,1699916340,45,0,3300,S_TUC,NaN,FRIGO,NAVES,NaN,NaN,NaN,NaN,NaN,NaN,14.273747,50.137567,POINT (14.27375 50.13757),1
2,006ZABOJ,006ZABOJ-TROCH,S_TUC,FM Česká s.r.o.,FM Logistic,0.0,13000,540,0,NaN,2023-11-12 22:00:00,2023-11-13 23:59:00,1699822800,1699916340,45,0,3300,S_TUC,NaN,FRIGO,NAVES,NaN,NaN,NaN,NaN,NaN,NaN,14.273747,50.137567,POINT (14.27375 50.13757),2
3,004ZABOJ,004ZABOJ-HANSL,S_TUC,FM Česká s.r.o.,FM Logistic,0.0,20000,540,0,NaN,2023-11-12 22:00:00,2023-11-13 23:59:00,1699822800,1699916340,45,0,3300,S_TUC,NaN,FRIGO,NAVES,NaN,NaN,NaN,NaN,NaN,NaN,14.273747,50.137567,POINT (14.27375 50.13757),3
4,TESCO,TESCO-TESCO,S_TUC,FM Česká s.r.o.,FM Logistic,0.0,3016,540,0,NaN,2023-11-12 22:00:00,2023-11-13 23:59:00,1699822800,1699916340,45,0,3300,S_TUC,NaN,FRIGO,NAVES,TESCO,NaN,NaN,NaN,NaN,NaN,14.273747,50.137567,POINT (14.27375 50.13757),4


In [9]:
vehicles = vehicles.drop_duplicates(subset=['vehicle_id'])

In [10]:
shipments['pickup_location_id']   = DEPOT_ID
shipments['drop_off_location_id'] = DEPOT_ID

In [11]:
problem = FleetRouting(
    data={'shipments': shipments, 'depots': depots, 'vehicles': vehicles}, 
    expr=('depot_id', '==', DEPOT_ID),
    populate_polylines=True,
    utc_offset=1,
    label=DEPOT_LABEL
)

problem.describe()


Summary:
    88 shipments
     1 depots
    20 vehicles


<FleetRouting>

In [12]:
problem.tag_delimiter

'-'

In [13]:
problem.build_scenario().save_scenario(filename='scenario.json')

<FleetRouting>

In [14]:
problem.load_solution(filename='scenario.zip')

<FleetRouting>

In [15]:
problem.plot_scenario(basemap_label='positron')

In [16]:
problem.plot_solution()

In [17]:
problem.metrics

{'total_cost': 53819.46781,
 'number_of_vehicles_used': 10,
 'number_of_skipped_mandatory_shipments': 0,
 'travel_time': 2077,
 'travel_time_text': '34:37:44',
 'idle_time': 207,
 'idle_time_text': '03:27:47',
 'delay_duration': 0,
 'delay_duration_text': '00:00:00',
 'break_duration': 0,
 'break_duration_text': '00:00:00',
 'service_time': 2384,
 'service_time_text': '39:44:00',
 'total_duration': 4669,
 'total_duration_text': '77:49:31',
 'total_distance': 2322.482,
 'total_distance_text': '2,322.5 km'}

In [18]:
problem.total_distance

2322.482

In [19]:
problem.number_of_vehicles_used

10

In [20]:
problem.visits.head()

,gmapsro_id,shipment_name,is_pickup,vehicle_id,vehicle_name,assigned_datetime,visit_name,assigned_route_order_id,assigned_date,assigned_time,id,location_id,org_name,depot_id,depot_name,shipment_type,period,date_of_delivery,time_window_start,time_window_end,duration,floor_pallets,tags,forbidden_tags,allowed_vehicles,input_address,input_postal_code,input_city,input_country_code,formatted_address,street_number,street,address,city,postal_code,admin_area_level_2,admin_area_level_1,country,country_code,place_id,place_name,place_type,place_main_type,place_URL,website,phone,email,monday,tuesday,wednesday,thursday,friday,saturday,sunday,location_type,confidence,maps_URL,longitude,latitude,geometry,start_datetime,end_datetime,start_timestamp,end_timestamp,pickup_location_id,drop_off_location_id
0,0,17.listopadu_70800,False,8,4AM 1209-Škrabal,2023-11-13T07:45:53.000Z,R574+93 708 00 Ostrava-Poruba Czechia,22,2023-11-13,07:45:53,14390261_D,17.listopadu_70800,FM Logistic,S_OLO,HUB Olomouc,depot_pickup_and_delivery,2023-11-13,2023-11-13,06:00:00,18:00:00,15,100,S_OLO,,4S5 6332|7M7 3641|8AL 2282|7AF 6001|7Z2 3406|1BS 2104|4M5 6808|6S1 0664|2M7 1261|6M8 2032|6M9 91...,17. listopadu,708 00,Ostrava,CZ,"R574+93, 708 00 Ostrava-Poruba, Czechia",NaN,R574+93 708 00 Ostrava-Poruba Czechia,R574+93 708 00 Ostrava-Poruba Czechia,Ostrava 8,708 00,Ostrava-City District,Moravian-Silesian Region,Czechia,cz,ChIJrbeS837nE0cRZxaUbRdvOC4,Rohlik.cz Sklad Ostrava,"['storage', 'point_of_interest', 'establishment']",storage,https://maps.google.com/?cid=3330534070853834343,http://www.rohlik.cz/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GEOMETRIC_CENTER,0.68,https://www.google.com/maps/search/?api=1&query=49.813385%2C18.155238&query_place_id=ChIJrbeS837...,18.155238,49.813385,POINT (18.15524 49.81339),2023-11-13 06:00:00,2023-11-13 18:00:00,1699851600,1699894800,S_OLO,S_OLO
1,0,17.listopadu_70800,True,8,4AM 1209-Škrabal,2023-11-13T03:15:30.000Z,HUB Olomouc,6,2023-11-13,03:15:30,14390261_D,17.listopadu_70800,FM Logistic,S_OLO,HUB Olomouc,depot_pickup_and_delivery,2023-11-13,2023-11-13,06:00:00,18:00:00,15,100,S_OLO,,4S5 6332|7M7 3641|8AL 2282|7AF 6001|7Z2 3406|1BS 2104|4M5 6808|6S1 0664|2M7 1261|6M8 2032|6M9 91...,17. listopadu,708 00,Ostrava,CZ,"R574+93, 708 00 Ostrava-Poruba, Czechia",NaN,R574+93 708 00 Ostrava-Poruba Czechia,R574+93 708 00 Ostrava-Poruba Czechia,Ostrava 8,708 00,Ostrava-City District,Moravian-Silesian Region,Czechia,cz,ChIJrbeS837nE0cRZxaUbRdvOC4,Rohlik.cz Sklad Ostrava,"['storage', 'point_of_interest', 'establishment']",storage,https://maps.google.com/?cid=3330534070853834343,http://www.rohlik.cz/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GEOMETRIC_CENTER,0.68,https://www.google.com/maps/search/?api=1&query=49.813385%2C18.155238&query_place_id=ChIJrbeS837...,18.155238,49.813385,POINT (18.15524 49.81339),2023-11-13 06:00:00,2023-11-13 18:00:00,1699851600,1699894800,S_OLO,S_OLO
2,1,sportovni_60200,False,16,8AL 2282-Lasovská,2023-11-13T08:33:14.000Z,Sportovní 594/21 602 00 Brno-Královo Pole Czechia,10,2023-11-13,08:33:14,14390595_D,sportovni_60200,FM Logistic,S_OLO,HUB Olomouc,depot_pickup_and_delivery,2023-11-13,2023-11-13,07:00:00,16:00:00,15,40,S_OLO,NAVES,4S5 6332|7M7 3641|8AL 2282|1BS 2104|4M5 6808|6S1 0664|2M7 1261|6M8 2032|6M9 9172|6S1 0663|6S1 06...,Sportovni,602 00,Brno - Ponava,CZ,"Sportovní 594/21, 602 00 Brno-Královo Pole, Czechia",594/21,Sportovní,594/21 Sportovní,NaN,602 00,Brno-město,Jihomoravský kraj,Czechia,cz,ChIJa1tGCEuVEkcRlKK7SQIrUBo,Kaufland Brno-Ponava,"['supermarket', 'grocery_or_supermarket', 'store', 'point_of_interest', 'food', 'establishment']",supermarket,https://maps.google.com/?cid=1896062731949941396,https://prodejny.kaufland.cz/?cid=CZ:loc:uberall_1681738949195,+420 549 211 222,NaN,07:00-22:00,07:00-22:00,07:00-22:00,07:00-22:00,07:00-22:00,07:00-22:00,07:00-22:00,ROOFTOP,0.68,https://www.google.com/maps/search/?api=1&query=49.215383%2C16.604088&query_place_id=ChIJa1tGCEu...,16.604088,49.215383,POINT (16.60409 49.21538),2023-11-1

In [21]:
problem.visits.tail()

,gmapsro_id,shipment_name,is_pickup,vehicle_id,vehicle_name,assigned_datetime,visit_name,assigned_route_order_id,assigned_date,assigned_time,id,location_id,org_name,depot_id,depot_name,shipment_type,period,date_of_delivery,time_window_start,time_window_end,duration,floor_pallets,tags,forbidden_tags,allowed_vehicles,input_address,input_postal_code,input_city,input_country_code,formatted_address,street_number,street,address,city,postal_code,admin_area_level_2,admin_area_level_1,country,country_code,place_id,place_name,place_type,place_main_type,place_URL,website,phone,email,monday,tuesday,wednesday,thursday,friday,saturday,sunday,location_type,confidence,maps_URL,longitude,latitude,geometry,start_datetime,end_datetime,start_timestamp,end_timestamp,pickup_location_id,drop_off_location_id
171,85,hranicka1467_75131,False,8,4AM 1209-Škrabal,2023-11-13T08:37:40.000Z,Hranická 1467 Město 751 31 Lipník nad Bečvou Czechia,23,2023-11-13,08:37:40,14392049_D,hranicka1467_75131,FM Logistic,S_OLO,HUB Olomouc,depot_pickup_and_delivery,2023-11-13,2023-11-13,06:00:00,14:00:00,40,100,S_OLO,,4S5 6332|7M7 3641|8AL 2282|7AF 6001|7Z2 3406|1BS 2104|4M5 6808|6S1 0664|2M7 1261|6M8 2032|6M9 91...,Hranicka 1467,751 31,Lipnik nad Becvou,CZ,"Hranická 1467, Město, 751 31 Lipník nad Bečvou, Czechia",NaN,Hranická,Hranická,Lipník nad Bečvou,751 31,Přerov,Olomoucký kraj,Czechia,cz,ChIJ2UDimniwE0cRjoERrLAj7Lc,1467,['street_address'],street_address,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROOFTOP,0.83,https://www.google.com/maps/search/?api=1&query=49.528425%2C17.607268&query_place_id=ChIJ2UDimni...,17.607268,49.528425,POINT (17.60727 49.52843),2023-11-13 06:00:00,2023-11-13 14:00:00,1699851600,1699880400,S_OLO,S_OLO
172,86,navyhone3682_69501,True,18,7M7 3641-JANCIK,2023-11-13T04:02:04.000Z,HUB Olomouc,3,2023-11-13,04:02:04,14392062_D,navyhone3682_69501,FM Logistic,S_OLO,HUB Olomouc,depot_pickup_and_delivery,2023-11-13,2023-11-13,06:00:00,12:00:00,15,200,S_OLO,,4S5 6332|7M7 3641|8AL 2282|7AF 6001|7Z2 3406|1BS 2104|4M5 6808|6S1 0664|2M7 1261|6M8 2032|6M9 91...,Na Vyhone 3682,695 01,Hodonin,CZ,"Na Výhoně 3682, 695 01 Hodonín-Hodonín 1, Czechia",NaN,Na Výhoně,Na Výhoně,Hodonín 1,695 01,Okres Hodonín,Jihomoravský kraj,Czechia,cz,ChIJcYlfX68vE0cRwqd9N1Rh_7o,Wholesale TIME - Ing.Petr Čechovský s.f.o.,"['point_of_interest', 'food', 'establishment']",point_of_interest,https://maps.google.com/?cid=13474595624451942338,http://www.cas-hod.cz/,+420 518 344 822,NaN,08:00-18:00,08:00-18:00,08:00-18:00,08:00-18:00,08:00-20:00,08:00-12:00,08:00-12:00,ROOFTOP,0.75,https://www.google.com/maps/search/?api=1&query=48.867347%2C17.138630&query_place_id=ChIJcYlfX68...,17.138630,48.867347,POINT (17.13863 48.86735),2023-11-13 06:00:00,2023-11-13 12:00:00,1699851600,1699873200,S_OLO,S_OLO
173,86,navyhone3682_69501,False,18,7M7 3641-JANCIK,2023-11-13T07:35:09.000Z,Na Výhoně 3682 695 01 Hodonín-Hodonín 1 Czechia,14,2023-11-13,07:35:09,14392062_D,navyhone3682_69501,FM Logistic,S_OLO,HUB Olomouc,depot_pickup_and_delivery,2023-11-13,2023-11-13,06:00:00,12:00:00,15,200,S_OLO,,4S5 6332|7M7 3641|8AL 2282|7AF 6001|7Z2 3406|1BS 2104|4M5 6808|6S1 0664|2M7 1261|6M8 2032|6M9 91...,Na Vyhone 3682,695 01,Hodonin,CZ,"Na Výhoně 3682, 695 01 Hodonín-Hodonín 1, Czechia",NaN,Na Výhoně,Na Výhoně,Hodonín 1,695 01,Okres Hodonín,Jihomoravský kraj,Czechia,cz,ChIJcYlfX68vE0cRwqd9N1Rh_7o,Wholesale TIME - Ing.Petr Čechovský s.f.o.,"['point_of_interest', 'food', 'establishment']",point_of_interest,https://maps.google.com/?cid=13474595624451942338,http://www.cas-hod.cz/,+420 518 344 822,NaN,08:00-18:00,08:00-18:00,08:00-18:00,08:00-18:00,08:00-20:00,08:00-12:00,08:00-12:00,ROOFTOP,0.75,https://www.google.com/maps/search/?api=1&query=48.867347%2C17.138630&query_place_id=ChIJcYlfX68...,17.138630,48.867347,POINT (17.13863 48.86735),2023-11-13 06:00:00,2023-11-13 12:00:00,1699851600,1699873200,S_OLO,S_OLO
174,87,jihlavská672/31_62500,True,12,1BS 2104-Stratil,2023-11-13T06:23:58.000Z,HUB Olomouc,6,2023-11-13,06:23:58,143

In [22]:
problem.routes.head(1)

,vehicle_id,vehicle_name,vehicle_start_time,vehicle_end_time,number_of_shipments,number_of_stops,total_cost,travel_time,idle_time,delay_duration,break_duration,visit_duration,total_duration,distance,max_loads,route_polyline,geometry,floor_pallets,distance_text,travel_time_text,idle_time_text,delay_duration_text,break_duration_text,visit_duration_text,total_duration_text,total_cost_text
0,0,4AJ 1413-NOSEK,2023-11-13T03:06:39.000Z,2023-11-13T11:48:42.000Z,8,16,4500.0,262,0,0,0,260,522,328.663,{'floor_pallets': {'amount': '3060'}},{'points': 'wjpmHiu_hBb@bA]f@BDHPrDfIp@xAjCxFBDjB`E\v@Rd@bBpDb@~@DJ@Cx@oAz@mA@??I^k@hBoCVZNPl@p@...,"MULTILINESTRING ((17.20677 49.56860, 17.20643 49.56842, 17.20623 49.56857, 17.20620 49.56855, 17...",3060,328.7 km,00:04:22,00:00:00,00:00:00,00:00:00,00:04:20,00:08:42,"4,500.0 €"


In [23]:
problem.vehicles.head(1)

,vehicle_id,vehicle_name,depot_id,depot_name,org_name,cost_per_kilometer,fixed_cost,route_duration_limit,route_duration_limit_soft_max,route_duration_limit_cost_per_hour_after_soft_max,start_datetime,end_datetime,start_timestamp,end_timestamp,depot_service_time,allowed_overtime,floor_pallets,S_TUC,S_OLO,FRIGO,NAVES,TESCO,CELO,SOLO,BOHEMIA_ONLY,DODAVKA,KINDL,longitude,latitude,geometry,gmapsro_id
0,4AJ 1413,4AJ 1413-NOSEK,S_OLO,HUB Olomouc,FM Logistic,0.0,4500,540,0,NaN,2023-11-12 22:00:00,2023-11-13 23:59:00,1699822800,1699916340,45,0,3800,NaN,S_OLO,NaN,NAVES,NaN,NaN,NaN,NaN,NaN,NaN,17.206987,49.568437,POINT (17.20699 49.56844),0
